## Вычисление гомологий

In [ ]:
using Pkg
Pkg.update("NPZ")

### **Конвертер из .txt в .npz с эмбеддингами (отдельно)**

(Здесь художественные эмбеддинги)

In [ ]:
using DelimitedFiles
using NPZ

function convert_embeddings_to_npz(input_file, output_file)
    data = readdlm(input_file, ' ', String)
    
    #извлечение слов и векторов
    words = data[:, 1]
    vectors = tryparse.(Float64, data[:, 2:end])

    dims = size(vectors, 2)
    println("Размерность эмбеддингов: ", dims)
    println("Количество слов: ", length(words))
    
    if any(isnan.(vectors)) || any(isinf.(vectors)).
        error("Обнаружены NaN или Inf значения в эмбеддингах!")
    end
    
    # создание словаря
    embeddings_dict = Dict(words[i] => vectors[i, :] for i in 1:length(words))
    
    npzwrite(output_file, embeddings_dict)
    println("Файл успешно сохранен как: ", output_file)
end




convert_embeddings_to_npz("urmi_dictionary.txt", "urmi_embeddings.npz")

### **Построение семантического пространства**

#### **Ниже две диаграммы (семантического пространства и песистентности) для художественных текстов:**

In [6]:
using Ripserer
using LinearAlgebra
using Plots
using Distances
using NPZ
using Printf
using DelimitedFiles

# встроенный конвертер
function convert_embeddings_to_npz(input_file, output_file) 
    data = readdlm(input_file, ' ', String)
    words = data[:, 1]
    vectors = tryparse.(Float64, data[:, 2:end])
    
    dims = size(vectors, 2)
    println("Размерность эмбеддингов: ", dims)
    println("Количество слов: ", length(words))
    
    if any(isnan.(vectors)) || any(isinf.(vectors))
        error("Обнаружены NaN или Inf значения в эмбеддингах!")
    end
    
    # создание словаря
    embeddings_dict = Dict(words[i] => vectors[i, :] for i in 1:length(words))
    
    npzwrite(output_file, embeddings_dict)
    println("Файл успешно сохранен как: ", output_file)
end


#вычисление гомологий
function get_representatives(data, k, metric=Distances.euclidean)
    # удаление дубликатов, проверка на уникальность
    unique_data = unique(data)
    
    if length(unique_data) < 2
        error("Недостаточно уникальных точек для построения графа.")
    end
    
    # построение матрицы расстояний
    dists = pairwise(metric, unique_data)
    # проверка на наличие ребер
    if size(dists)[1] == 0 || size(dists)[2] == 0
        error("Матрица расстояний пуста.")
    end
    
    # Вычисление п.г
    diagram_cocycles = ripserer(dists; reps=true)
    
    k_ = min(size(diagram_cocycles[2])[1], k)
    
    most_persistent_co = diagram_cocycles[2][end-k_+1:end]
    
    filtration = diagram_cocycles[2].filtration
    cycles = [reconstruct_cycle(filtration, mpc) for mpc in most_persistent_co]
    
    @printf("Выбрано %d циклов из %d точек\n", k_, length(unique_data))
    return diagram_cocycles, cycles
end

function save_intervals(filename::String, intervals)
    bd_mat = isempty(intervals) ? Array{Float64}(undef, 0, 2) :
        reduce(vcat, [reshape([bd[1], bd[2]], 1, 2) for bd in intervals])
    npzwrite(filename, bd_mat)
end


function process_data(
    data_path::String = "urmi_embeddings.npz",
    metric = Distances.cosine_dist # косинусную метрику берём
)
    # загрузка всего
    vars = npzread(data_path)
    words = collect(keys(vars))
    data = [vars[w] for w in words]
    @printf("Всего загружено %d слов (точек)\n", length(data))
    @printf("Размерность эмбеддингов: %d\n", length(data[1]))

    # доработка данных
    data = [Float64.(x) for x in data]  # тип Float64
    data = [x .+ 1e-9*randn(size(x)) for x in data]  # добавление небольшого шума

    # визуализация первых 2 или 3 компонент (3х)
    if length(data[1]) >= 2
        coords = hcat(data...)
        if size(coords, 1) >= 3
            p = scatter3d(coords[1,:], coords[2,:], coords[3,:], 
                     markersize=2, legend=false, title="3D Embeddings Projection")
        else
            p = scatter(coords[1,:], coords[2,:], 
                   markersize=2, legend=false, title="2D Embeddings Projection")
        end
        display(p)
    end
    diagram, cycles = get_representatives(data, 500, metric)
    
    plt = plot(diagram)
    display(plt)
    
    save_intervals("h0_results_urmi.npy", diagram[1])
    save_intervals("h1_results_urmi.npy", diagram[2])
    
    scycles = [reduce(vcat, [reshape(collect(vertices(sx)), 1, :) for sx in cycle]) for cycle in cycles]
    sscycles = isempty(scycles) ? Array{Int64}(undef, 0, 1) : reduce(vcat, scycles)
    sscycles = Matrix{Int64}(sscycles)
    
    npzwrite("urmi_word_holes.npy", sscycles)
    return diagram, cycles
end

# Конвертер
convert_embeddings_to_npz("urmi_dictionary.txt", "urmi_embeddings.npz")

# процесс...
diagram, cycles = process_data("urmi_embeddings.npz")

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/kuklelik/Jupyter/In[6]#1:6\[90mIn[6]:1:6[0;0m]8;;\
chmod[48;2;120;70;70m u+w /home/kukelik/Jupyter/[0;0m
[90m#    └─────────────────────────┘ ── [0;0m[91mextra tokens after end of expression[0;0m

#### **Ниже две диаграммы (семантического пространства и песистентности) для газетных текстов (без именованными сущностей):**


In [ ]:
using Ripserer
using LinearAlgebra
using Plots
using Distances
using NPZ
using Printf
using DelimitedFiles

function convert_embeddings_to_npz(input_file, output_file) 
    data = readdlm(input_file, ' ', String)
    words = data[:, 1]
    vectors = tryparse.(Float64, data[:, 2:end])
    
    dims = size(vectors, 2)
    println("Размерность эмбеддингов: ", dims)
    println("Количество слов: ", length(words))
    
    if any(isnan.(vectors)) || any(isinf.(vectors))
        error("Обнаружены NaN или Inf значения в эмбеддингах!")
    end
    
    # создание словаря
    embeddings_dict = Dict(words[i] => vectors[i, :] for i in 1:length(words))
    
    npzwrite(output_file, embeddings_dict)
    println("Файл успешно сохранен как: ", output_file)
end


#вычисление гомологий
function get_representatives(data, k, metric=Distances.euclidean)
    # удаление дубликатов, проверка на уникальность
    unique_data = unique(data)
    
    if length(unique_data) < 2
        error("Недостаточно уникальных точек для построения графа.")
    end
    
    # построение матрицы расстояний
    dists = pairwise(metric, unique_data)
    # проверка на наличие ребер
    if size(dists)[1] == 0 || size(dists)[2] == 0
        error("Матрица расстояний пуста.")
    end
    
    # Вычисление п.г
    diagram_cocycles = ripserer(dists; reps=true)
    
    k_ = min(size(diagram_cocycles[2])[1], k)
    
    most_persistent_co = diagram_cocycles[2][end-k_+1:end]
    
    filtration = diagram_cocycles[2].filtration
    cycles = [reconstruct_cycle(filtration, mpc) for mpc in most_persistent_co]
    
    @printf("Выбрано %d циклов из %d точек\n", k_, length(unique_data))
    return diagram_cocycles, cycles
end

function save_intervals(filename::String, intervals)
    bd_mat = isempty(intervals) ? Array{Float64}(undef, 0, 2) :
        reduce(vcat, [reshape([bd[1], bd[2]], 1, 2) for bd in intervals])
    npzwrite(filename, bd_mat)
end


function process_data(
    data_path::String = "urmi_embeddings.npz",
    metric = Distances.cosine_dist # косинусную метрику берём
)
    # загрузка всего
    vars = npzread(data_path)
    words = collect(keys(vars))
    data = [vars[w] for w in words]
    @printf("Всего загружено %d слов (точек)\n", length(data))
    @printf("Размерность эмбеддингов: %d\n", length(data[1]))

    # доработка данных
    data = [Float64.(x) for x in data]  # тип Float64
    data = [x .+ 1e-9*randn(size(x)) for x in data]  # добавление небольшого шума

    # визуализация первых 2 или 3 компонент (3х)
    if length(data[1]) >= 2
        coords = hcat(data...)
        if size(coords, 1) >= 3
            p = scatter3d(coords[1,:], coords[2,:], coords[3,:], 
                     markersize=2, legend=false, title="3D Embeddings Projection")
        else
            p = scatter(coords[1,:], coords[2,:], 
                   markersize=2, legend=false, title="2D Embeddings Projection")
        end
        display(p)
    end
    diagram, cycles = get_representatives(data, 500, metric)
    
    plt = plot(diagram)
    display(plt)
    # Сохранение результатов в текущую директорию
    save_intervals("h0_results_urmi_phot_new.npy", diagram[1])
    save_intervals("h1_results_urmi_photo_new.npy", diagram[2])
    
    scycles = [reduce(vcat, [reshape(collect(vertices(sx)), 1, :) for sx in cycle]) for cycle in cycles]
    sscycles = isempty(scycles) ? Array{Int64}(undef, 0, 1) : reduce(vcat, scycles)
    sscycles = Matrix{Int64}(sscycles)
    
    npzwrite("urmi_word_holes_photo_new.npy", sscycles)
    
    return diagram, cycles
end

# Преобразование текстового файла в npz (выполнить один раз)
convert_embeddings_to_npz("urmi_dictionary_photo_new.txt", "urmi_embeddings_photo_new.npz")

# Анализ данных
diagram, cycles = process_data("urmi_embeddings_photo_new.npz")

#### **Ниже две диаграммы (семантического пространства и песистентности) для комбинации (всех) текстов:**

In [ ]:
using Ripserer
using LinearAlgebra
using Plots
using Distances
using NPZ
using Printf
using DelimitedFiles

function convert_embeddings_to_npz(input_file, output_file)
    data = readdlm(input_file, ' ', String)
    words = data[:, 1]
    vectors = tryparse.(Float64, data[:, 2:end])
    
    dims = size(vectors, 2)
    println("Размерность эмбеддингов: ", dims)
    println("Количество слов: ", length(words))
    
    if any(isnan.(vectors)) || any(isinf.(vectors))
        error("Обнаружены NaN или Inf значения в эмбеддингах!")
    end
    
    embeddings_dict = Dict(words[i] => vectors[i, :] for i in 1:length(words))
    
    npzwrite(output_file, embeddings_dict)
    println("Файл успешно сохранен как: ", output_file)
end

function get_representatives(data, k, metric=Distances.euclidean)
    unique_data = unique(data)
    
    if length(unique_data) < 2
        error("Недостаточно уникальных точек для построения графа.")
    end
    
    # построение матрицы расстояний
    dists = pairwise(metric, unique_data)
    
    # проверка на наличие ребер
    if size(dists)[1] == 0 || size(dists)[2] == 0
        error("Матрица расстояний пуста.")
    end
    
    # Вычисление п.г.
    diagram_cocycles = ripserer(dists; reps=true)
    
    k_ = min(size(diagram_cocycles[2])[1], k)
    
    most_persistent_co = diagram_cocycles[2][end-k_+1:end]
    
    filtration = diagram_cocycles[2].filtration
    cycles = [reconstruct_cycle(filtration, mpc) for mpc in most_persistent_co]
    
    @printf("Выбрано %d циклов из %d точек\n", k_, length(unique_data))
    return diagram_cocycles, cycles
end

function save_intervals(filename::String, intervals)
    bd_mat = isempty(intervals) ? Array{Float64}(undef, 0, 2) :
        reduce(vcat, [reshape([bd[1], bd[2]], 1, 2) for bd in intervals])
    npzwrite(filename, bd_mat)
end

function process_data(
    data_path::String = "urmi_embeddings.npz",
    metric = Distances.cosine_dist
)
    # загрузка данных
    vars = npzread(data_path)
    words = collect(keys(vars))
    data = [vars[w] for w in words]
    
    @printf("Всего загружено %d слов (точек)\n", length(data))
    @printf("Размерность эмбеддингов: %d\n", length(data[1]))

    # доработка данных
    data = [Float64.(x) for x in data]  
    data = [x .+ 1e-9*randn(size(x)) for x in data]  # шум

    # визуализация первых 2-3 компонент (3x)
    if length(data[1]) >= 2
        coords = hcat(data...)
        if size(coords, 1) >= 3
            p = scatter3d(coords[1,:], coords[2,:], coords[3,:], 
                     markersize=2, legend=false, title="3D Embeddings Projection")
        else
            p = scatter(coords[1,:], coords[2,:], 
                   markersize=2, legend=false, title="2D Embeddings Projection")
        end
        display(p)
    end

    diagram, cycles = get_representatives(data, 500, metric)
    
    plt = plot(diagram)
    display(plt)
    
    save_intervals("h0_results_urmi_general.npy", diagram[1])
    save_intervals("h1_results_urmi_photo_new.npy", diagram[2])
    
    scycles = [reduce(vcat, [reshape(collect(vertices(sx)), 1, :) for sx in cycle]) for cycle in cycles]
    sscycles = isempty(scycles) ? Array{Int64}(undef, 0, 1) : reduce(vcat, scycles)
    sscycles = Matrix{Int64}(sscycles)
    
    npzwrite("urmi_word_holes_general.npy", sscycles)
    
    return diagram, cycles
end

convert_embeddings_to_npz("urmi_dictionary_general.txt", "urmi_embeddings_general.npz")

diagram, cycles = process_data("urmi_embeddings_general.npz")

#### **Код для нахождения 20 самых персистентных "дыр" и записи их в файл**

здесь у художественных текстов:

In [ ]:
using Ripserer
using Distances
using LinearAlgebra
using NPZ
using Printf

function get_representatives(data, k; metric=Distances.cosine_dist)
    dgms = ripserer(data; reps=true, metric=metric)
    h1 = dgms[2]
    
    if isempty(h1)
        @printf("Не найдено ни одного цикла в H₁.\n")
        return dgms, []
    end
    
    k_ = min(length(h1), k)
    most_persistent = h1[end-k_+1:end]
    
    filtration = h1.filtration
    
    cycles = [reconstruct_cycle(filtration, mpc) for mpc in most_persistent]
    @printf("Найдено %d циклов (из H₁) на выборке из %d точек.\n", k_, length(data))
    
    return dgms, cycles
end

function save_holes_to_txt(cycles, embedding_keys, X, filename)
    open(filename, "w") do io
        if isempty(cycles)
            @warn "Список циклов H₁ пуст. Файл $(filename) будет пустым или содержать заметку."
            println(io, "Нет найденных 1-мерных дыр (H₁).")
            return
        end
        
        for (i, cycle) in enumerate(cycles)
            all_verts = Set{Int}()
            for simplex in cycle
                v_inds = vertices(simplex)
                union!(all_verts, v_inds)
            end
            
            hole_description = String[]
            for v in sort(collect(all_verts))
                word = embedding_keys[v]
                coords = X[v, :]
                coords_str = join(coords, " ")
                push!(hole_description, "$word")
            end
            
            line_str = join(hole_description, " | ")
            println(io, line_str)
        end
    end
    @printf("Данные о циклах H₁ сохранены в '%s'.\n", filename)
end
function main()
    npz_data = npzread("urmi_embeddings.npz")
    
    embedding_keys = collect(keys(npz_data))
    
    X = hcat([npz_data[k] for k in embedding_keys]...)'
    X = Matrix(X) 
    
    n_points = size(X, 1)
    data = [X[i, :] for i in 1:n_points]
    @info "Считано $n_points точек, размерность = $(size(X,2))."
    
    dgms, cycles = get_representatives(data, 20; metric=Distances.euclidean)
    
    save_holes_to_txt(cycles, embedding_keys, X, "holes_bounds_urmi1.txt")
end

main()

здесь общие "дыры" по всем текстам:

In [ ]:
using Ripserer
using Distances
using LinearAlgebra
using NPZ
using Printf

function get_representatives(data, k; metric=Distances.cosine_dist)
    dgms = ripserer(data; reps=true, metric=metric)
    h1 = dgms[2]
    
    if isempty(h1)
        @printf("Не найдено ни одного цикла в H₁.\n")
        return dgms, []
    end
    
    k_ = min(length(h1), k)
    most_persistent = h1[end-k_+1:end]
    
    filtration = h1.filtration
    
    cycles = [reconstruct_cycle(filtration, mpc) for mpc in most_persistent]
    @printf("Найдено %d циклов (из H₁) на выборке из %d точек.\n", k_, length(data))
    
    return dgms, cycles
end

function save_holes_to_txt(cycles, embedding_keys, X, filename)
    open(filename, "w") do io
        if isempty(cycles)
            @warn "Список циклов H₁ пуст. Файл $(filename) будет пустым или содержать заметку."
            println(io, "Нет найденных 1-мерных дыр (H₁).")
            return
        end
        
        for (i, cycle) in enumerate(cycles)
            all_verts = Set{Int}()
            for simplex in cycle
                v_inds = vertices(simplex)
                union!(all_verts, v_inds)
            end
            
            hole_description = String[]
            for v in sort(collect(all_verts))
                word = embedding_keys[v]
                coords = X[v, :]
                coords_str = join(coords, " ")
                push!(hole_description, "$word")
            end
            
            line_str = join(hole_description, " | ")
            println(io, line_str)
        end
    end
    @printf("Данные о циклах H₁ сохранены в '%s'.\n", filename)
end
function main()
    npz_data = npzread("urmi_embeddings_general.npz")
    
    embedding_keys = collect(keys(npz_data))
    
    X = hcat([npz_data[k] for k in embedding_keys]...)'
    X = Matrix(X) 
    
    n_points = size(X, 1)
    data = [X[i, :] for i in 1:n_points]
    @info "Считано $n_points точек, размерность = $(size(X,2))."
    
    dgms, cycles = get_representatives(data, 20; metric=Distances.euclidean)
    
    save_holes_to_txt(cycles, embedding_keys, X, "holes_bounds_urmi_general.txt")
end

main()